Straight copy of log_FE

In [1]:
import sys
import numpy as np
import pandas as pd

## Load DMS and store object

In [2]:
sys.path.append('../')

from data_prep import dm_saver, data_manager

dms = dm_saver.DM_Saver.load('../dms')

dml = data_manager.Data_Manager()

dml.dms2dm(dms)

## Locating constant pvm y-intervals
Dataset B and C has 'holes' in y_train, where pv_measurement is constant. Therefore we don't want to train on this data and can remove the corresponding rows for both X and Y

In [3]:
y_hole_length = 200
from helpers import find_const_interval

In [4]:
# TAKES NO TIME TO RUN
y_train_a_const_idx, ca = find_const_interval(dml.train_a, 'pv_measurement', y_hole_length)
print('y_train_a anomalies:',ca)
y_train_b_const_idx, cb = find_const_interval(dml.train_b, 'pv_measurement', y_hole_length)
print('y_train_b anomalies:',cb)
y_train_c_const_idx, cc = find_const_interval(dml.train_c, 'pv_measurement', y_hole_length)
print('y_train_c anomalies:',cc)


y_train_a anomalies: 2
y_train_b anomalies: 17
y_train_c anomalies: 15


### Deleting the rows where pvm is const

In [5]:
train_a = dml.train_a
train_b = dml.train_b
train_c = dml.train_c

In [6]:
date_forecast_a_const = dml.train_a.iloc[y_train_a_const_idx]['date_forecast']
date_forecast_a_const_values = date_forecast_a_const.values
train_a = train_a[~train_a['date_forecast'].isin(date_forecast_a_const_values)]

date_forecast_b_const = dml.train_b.iloc[y_train_b_const_idx]['date_forecast']
date_forecast_b_const_values = date_forecast_b_const.values
train_b = train_b[~train_b['date_forecast'].isin(date_forecast_b_const_values)]

date_forecast_c_const = dml.train_c.iloc[y_train_c_const_idx]['date_forecast']
date_forecast_c_const_values = date_forecast_c_const.values
train_c = train_c[~train_c['date_forecast'].isin(date_forecast_c_const_values)]


In [9]:
# By running this block, the dm train_a etc. will consist of combined x and y obs and est data

dml.train_a = train_a
dml.train_b = train_b
dml.train_c = train_c

## Normalizing the train data and saving the amplitude

In [10]:
pv_a = dml.train_a['pv_measurement']
pv_b = dml.train_b['pv_measurement']
pv_c = dml.train_c['pv_measurement']

print('A:\n',pv_a.describe())
print('B:\n',pv_b.describe())
print('C:\n',pv_c.describe())

A:
 count    34083.000000
mean       630.631711
std       1165.926622
min          0.000000
25%          0.000000
50%          5.940000
75%        678.150000
max       5733.420000
Name: pv_measurement, dtype: float64
B:
 count    29132.000000
mean        99.522841
std        203.946382
min         -0.000000
25%         -0.000000
50%          0.000000
75%         83.662500
max       1152.300000
Name: pv_measurement, dtype: float64
C:
 count    23214.000000
mean        87.265551
std        173.398144
min         -0.000000
25%          0.000000
50%          0.000000
75%         88.200000
max        999.600000
Name: pv_measurement, dtype: float64


### Mean value

In [11]:
dml.amplitude = np.array([pv_a.mean(), pv_b.mean(),pv_c.mean()])
dml.amplitude

array([630.63171053,  99.52284101,  87.26555096])

In [12]:
%store dml

Stored 'dml' (Data_Manager)
